<a href="https://colab.research.google.com/github/usamaeltmsah/HR-Bot-evaluation-model/blob/universal-senence-encoder/HR_Bot_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
%%capture
# Install the latest Tensorflow version.sentence
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

In [50]:
# load the universal sentence encoder model
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded


In [51]:
import tensorflow as tf
import tensorflow_hub as hub
import math
import numpy as np

class Evaulator:
  def __embed(self, input):
    return model(input)

  def __calculate_correlation(self, features):
    corrList = np.inner(features[:-1], features[-1])
    return np.max(corrList)

  def evalute_applicant_answer(self, model_answers, applicant_answer):
    answers = model_answers
    answers.append(applicant_answer)
    embedding_answers = self.__embed(answers)
    corr = self.__calculate_correlation(embedding_answers)
    """Returns the similarity scores"""
    return round(corr*10,2)

In [52]:
!pip install flask-ngrok
!pip install Wikipedia-API

  Created wheel for Wikipedia-API: filename=Wikipedia_API-0.5.4-cp36-none-any.whl size=13462 sha256=c6c480f0d46e165754015505567199be1b50e2fc47199c5419344b92aacb347d
  Stored in directory: /root/.cache/pip/wheels/bf/40/42/ba1d497f3712281b659dd65b566fc868035c859239571a725a
Successfully built Wikipedia-API


In [53]:
import wikipediaapi
from googlesearch import search
# from flask import render_template

class Parser:

    def __init__(self, skills):
        self.skills = skills

    def parse(self, sign):
        sentences = []
        for skill in self.skills:
            sentences.append(skill['name'].split())
        
        search_keys = []
        search_key = ''

        for sentence in sentences:
            for i in range(len(sentences)-1):
                search_key += sentence[i]
                search_key += sign
            search_key += sentence[-1]
            search_keys.append(search_key)
            search_key = ''

        return search_keys

class FeedBack:
    
    def __init__(self, data):
        self.skills = data['skills']
        self.question = data['question']
        self.token = data['token']

        

    #Done
    def wikipediaSearch(self):
        result = []
        wiki_wiki = wikipediaapi.Wikipedia('en')
        page_question = wiki_wiki.page(self.question)
        skills = self.skills
        
        for skill in skills:
            x = wiki_wiki.page(skill['name'])
            result.append(x.summary)

        result.append(page_question.summary)
        return result
    
    #Done
    def googleSearch(self):
        skills = self.skills
        question = self.question
        result = []

        for skill in skills:
            query = 'site:' + skill['site'] + ' ' + question + ' + ' + skill['name']
            for j in search(query, tld="com", num=10, stop=1, pause=2): 
                result.append(str(j)) 

        return result

    #Done
    def courseraSearch(self):
        p = Parser(self.skills)
        search_keys = p.parse('%20')
        links = []
        result = []
        for search_key in search_keys:
            link = 'https://www.coursera.org/search?query=' + search_key
            links.append(link)
        
        for link in links :
            result.append(link)

        return result

    #Done
    def udemySearch(self):
        p = Parser(self.skills)
        search_keys = p.parse('+')
        links = []
        result = []
        for search_key in search_keys:
            link = 'https://www.udemy.com/courses/search/?q=' + search_key
            links.append(link)
        
        for link in links :
            result.append(link)

        return result
    
    #Done
    def pluralsightSearch(self):
        p = Parser(self.skills)
        search_keys = p.parse('%20')
        links = []
        result = []
        for search_key in search_keys:
            link = 'https://www.pluralsight.com/search?q=' + search_key
            links.append(link)
        
        for link in links :
            result.append(link)

        return result

    #Done
    def lyndaSearch(self):
        p = Parser(self.skills)
        search_keys = p.parse('%20')
        links = []
        result = []
        for search_key in search_keys:
            link = 'https://www.lynda.com/search?q=' + search_key
            links.append(link)
        
        for link in links :
            result.append(link)

        return result

    def search(self):
        token = self.token 
        if token == "hr_bot_2019_2020":    
            result = []
            result.append(self.wikipediaSearch())
            result.append(self.googleSearch())
            result.append(self.courseraSearch())
            result.append(self.udemySearch())
            result.append(self.pluralsightSearch())
            result.append(self.lyndaSearch())
            
            return result

        else:
            return None

# p = Parser('machine learning andrew eg')
# print(    p.parseForCoursera())
if __name__ == "__main__":
    pass

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort, jsonify

evaluator = Evaulator()
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/')
def index():
    return "Machine Learning Models For HR-Bot"

@app.route('/evaluate', methods=['POST'])
def evaluation():
    if not request.json or not 'token' in request.json or not 'answers' in request.json or not 'applicant_answer' in request.json:
        abort(400)

    token = str(request.json["token"])

    if not(token == "hr_bot_2019_2020"):
        abort(404)
    else:
        model_answers = request.json['answers']
        applicant_answer = request.json['applicant_answer']
        score = evaluator.evalute_applicant_answer(model_answers, applicant_answer)
        print()
        return jsonify({'score': str(score)}), 200


# intput example : 
# {
#     "data" :
#     {
#         "question": "what is the desicion tree", 
#         "skills": 
#         [
#             {
#                 "name": "machine learning",
#                 "site": "www.datacamp.com"
#             },
#             {
#                 "name": "object oriented",
#                 "site":"www.datacamp.com"
#             }
#         ],
#         "token": "hr_bot_2019_2020"
#     }
# }
@app.route('/interview/feedback')
def feedback():
    if not request.json or not 'data' in request.json:
        abort(400)

    data = (request.json['data'])
    f = FeedBack(data=data)
    my_feedback = f.search()

    if my_feedback is None:
        abort(404)
        return "This should not be returned"
    else:
        return jsonify({'feedback': my_feedback}), 200

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a7fd1a2ee44d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 04:28:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 04:29:03] "POST /evaluate HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 04:30:22] "POST /evaluate HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 04:35:27] "POST /evaluate HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 04:37:28] "POST /evaluate HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 04:37:43] "POST /evaluate HTTP/1.1" 200 -


ERROR:__main__:Exception on /evaluate [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-56-c1d0ce09674a>", line 24, in evaluation
    score = evaluator.evalute_applicant_answer(model_answers, app

INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 06:09:23] "GET /evaluate HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2020 08:17:23] "POST /evaluate HTTP/1.1" 200 -
